In [1]:
import time
from bs4 import BeautifulSoup
from requests import get
import pandas as pd


In [2]:
artist = "Severina"
url_teksovinet  = "https://tekstovi.net/2,212,0.html"  #url of the artist on tekstovi.net
delay = 2  #add delay between requests

## Create empty df:
column_names = ["Artist", "Song", "Views", "Lyrics", "Url"]
df = pd.DataFrame(columns = column_names)

response = get(url_teksovinet)
html_soup = BeautifulSoup(response.text, 'html.parser')
songs = html_soup.find_all('p', class_='artLyrList')


for index, song in enumerate(songs):
    song_name = song.findChild("a").text
    song_url = song.findChild("a")["href"]

    url = 'https://tekstovi.net/'+song_url

    response = get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')

    song_lyrics = html_soup.find_all('p', class_='lyric')
    lyrics = [l.text for l in song_lyrics]
    lyrics = " ".join(lyrics)
    views_text = html_soup.find('p', class_='lyric_impressions').text
    views_count = int("".join([c for c in views_text if c.isdigit()]))
    new_entry = pd.DataFrame([[artist, song_name, views_count, lyrics, url]], columns=column_names)
    df = df.append(new_entry, ignore_index=True)
    print("Successfully found lyrics for song " + song_name + " by " + artist)
    time.sleep(delay)



Successfully found lyrics for song 'Ko to pita by Severina
Successfully found lyrics for song 'Ocu s tobon zivit' by Severina
Successfully found lyrics for song Adam i Seva by Severina
Successfully found lyrics for song Adio, ljube by Severina
Successfully found lyrics for song Aj, aj, aj, aj by Severina
Successfully found lyrics for song Ajde, ajde zlato moje by Severina
Successfully found lyrics for song Ako odes ljubavi by Severina
Successfully found lyrics for song Alcatraz by Severina
Successfully found lyrics for song Ante by Severina
Successfully found lyrics for song Baby, baby by Severina
Successfully found lyrics for song Bambola by Severina
Successfully found lyrics for song Bijela vrana by Severina
Successfully found lyrics for song Bojate bane buski by Severina
Successfully found lyrics for song Bolna ti bolujem by Severina
Successfully found lyrics for song Boze moj by Severina
Successfully found lyrics for song Brad Pitt by Severina
Successfully found lyrics for song Bra

In [3]:
df

,Artist,Song,Views,Lyrics,Url
0,Severina,'Ko to pita,93225,"Falit ces mi, faliti\r\nali necu zaliti\r\nsan...","https://tekstovi.net/2,212,27546.html"
1,Severina,'Ocu s tobon zivit',3196,Stvorija te Bog samo mog da te volim\r\na nebo...,"https://tekstovi.net/2,212,58175.html"
2,Severina,Adam i Seva,69814,"Veceras ja ulazim u klub\r\nprva macka, plesna...","https://tekstovi.net/2,212,4419.html"
3,Severina,"Adio, ljube",23254,Ja sam tebi sve na svijetu vjerovala\r\nja sam...,"https://tekstovi.net/2,212,11661.html"
4,Severina,"Aj, aj, aj, aj",25595,U mojoj ulici jutro donose ptice\r\ni osmijehe...,"https://tekstovi.net/2,212,11653.html"
...,...,...,...,...,...
156,Severina,Zaustavite tramvaj,12872,"Vrela noc je, grad spava\r\ntramvaj vozi zadnj...","https://tekstovi.net/2,212,11674.html"
157,Severina,"Zdravo, Marijo",107654,Rakija se lijeci vinom\r\nmarihuana kokainom\r...,"https://tekstovi.net/2,212,26264.html"
158,Severina,"Zime, ljeta, jeseni",16595,Kada oci otvorim\r\nti si prvo sto mi padne na...,"https://tekstovi.net/2,212,11603.html"
159,Severina,"Zivila nan jubav (feat. G.Karan,Daniela,Giulia...",526,Zivila nan jubav\r\nIvana mog\r\ndicu ce nan d...,"https://tekstovi.net/2,212,62264.html"


In [9]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By

import numpy as np


In [10]:
def get_lyricstranslate(driver):
    
    driver.find_element(By.PARTIAL_LINK_TEXT, 'lyricstranslate.com').send_keys(Keys.CONTROL + Keys.RETURN)
    time.sleep(0.5)
    window_handles = driver.window_handles
    driver.switch_to.window(window_name=window_handles[1])    
    time.sleep(1)


    try: driver.find_element(By.XPATH, "//*[contains(text(), 'AGREE')]").click()
    except NoSuchElementException: pass
    time.sleep(0.5)
    
    try: driver.find_element(By.PARTIAL_LINK_TEXT, 'English').click()
    except NoSuchElementException: pass
    time.sleep(0.5)

    try: driver.find_element(By.PARTIAL_LINK_TEXT, 'Click to see the original lyrics').click()
    except NoSuchElementException: pass
    time.sleep(0.5)
    
    
    try:
        original_lyrics_raw = driver.find_element(By.CLASS_NAME, 'song-node-text')
        original_title = original_lyrics_raw.find_element(By.CLASS_NAME, 'title-h2').text
        par_class = original_lyrics_raw.find_elements(By.CLASS_NAME, 'par')
        original_lyrics = ". ".join([t.text for t in par_class])
        original_lyrics = original_lyrics.replace("*", "")
        try:
          copyright = original_lyrics_raw.find_element(By.CLASS_NAME, 'copyrighttext').text.split("\n")
          writer_info = [c for c in copyright if "Writer" in c]
          if len(writer_info)>0:
            writer = writer_info[0].split(": ")[1]
          else: 
            print("Couldn't find writer info")
            writer = np.nan
    
        except NoSuchElementException: 
          print("Couldn't find writer info")
          writer = np.nan


    except NoSuchElementException:
        print("Couldn't find original lyrics")
        original_title = np.nan
        original_lyrics = np.nan
        writer = np.nan

    try:
        eng_lyrics_raw = driver.find_element(By.CLASS_NAME, 'translate-node-text')
        eng_title = eng_lyrics_raw.find_element(By.CLASS_NAME, 'title-h2').text
        par_class = eng_lyrics_raw.find_elements(By.CLASS_NAME, 'par')
        eng_lyrics = ". ".join([t.text for t in par_class])
        eng_lyrics = eng_lyrics.replace("*", "")
    except NoSuchElementException:
        print("Couldn't find english lyrics")
        eng_title = np.nan
        eng_lyrics = np.nan



    try:
        song_info = driver.find_element(By.CLASS_NAME, 'song-node-info')
        album_name = song_info.find_element(By.XPATH, "//*[contains(text(), 'Album')]").text
        if len(album_name)>0:
            album_name = album_name.split(": ")
            if len(album_name)>1:
              album_name = album_name[1]
            else:
              print("Couldn't extract album info from string:")
              print(album_name)
        else: 
            print("Couldn't find album info")
            album_name =np.nan
    except NoSuchElementException: 
        print("Couldn't find album info")
        album_name = np.nan

    
    song_url = driver.current_url
    driver.close()
    driver.switch_to.window(window_name=window_handles[0])
    
    
    return writer, original_title, eng_title, album_name, original_lyrics, eng_lyrics, song_url

In [11]:
artist_name = "Severina"
song_name = "Adam i Seva"

ser = Service()
op = webdriver.ChromeOptions()
op.add_argument('--headless')
op.add_argument('--no-sandbox')
op.add_argument('--disable-dev-shm-usage')

 
driver = webdriver.Chrome(service=ser, options=op)
driver.get("https://google.com")
time.sleep(delay)
    
try: driver.find_element(By.ID, 'L2AGLb').click()  ##Accepting the cookies if necessary
except: pass

try: driver.find_element(By.XPATH, "//*[contains(text(), 'English')]").click()  ##Selecting language if necessary
except: pass

        
try:
    #search google
    search_query = artist_name+" "+song_name+ " english lyricstranslate"
    driver.find_element(By.NAME, 'q').clear()
    search_box = driver.find_element(By.NAME, 'q')
    search_box.send_keys(search_query)
    search_box.send_keys(Keys.RETURN)
    time.sleep(3)
    
    writer, original_title, eng_title, album_name, original_lyrics, eng_lyrics, song_url = get_lyricstranslate(driver)
    print(f"Succesfully found information for {song_name} from {artist_name}")
    time.sleep(3)

except NoSuchElementException:  print(f"Couldn't find lyrics for song {song_name} by {artist_name} from lyricstranslate.com")
    


Succesfully found information for Adam i Seva from Severina


In [13]:
dir(driver)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_file_detector',
 '_get_cdp_details',
 '_is_remote',
 '_mobile',
 '_shadowroot_cls',
 '_switch_to',
 '_unwrap_value',
 '_web_element_cls',
 '_wrap_value',
 'add_cookie',
 'application_cache',
 'back',
 'bidi_connection',
 'capabilities',
 'caps',
 'close',
 'command_executor',
 'create_options',
 'create_web_element',
 'current_url',
 'current_window_handle',
 'delete_all_cookies',
 'delete_cookie',
 'delete_network_conditions',
 'desired_capabilities',
 'error_handler',
 'execute',
 'execute_async_script',
 'execute_cdp_cmd',
 'execute_script',
 'file_dete

In [8]:
#wrong_rows = check_row_similariy(df, "Lyrics_tekstovinet", "Lyrics_lyricstranslate", threshold=0.7)


'Severgreen'